In [34]:
import torch
import sentencepiece as spm
from model import MiniGPT


In [35]:
sp = spm.SentencePieceProcessor()
sp.load("C:/Users/aksha/OneDrive/Desktop/llm/tokenizer/tokenizer.model")

def encode(text):
    return sp.encode(text)

def decode(tokens):
    return sp.decode(tokens)


In [74]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

VOCAB_SIZE = 8000
BLOCK_SIZE = 128   # ← THIS WAS MISSING

model = MiniGPT().to(DEVICE)

ckpt = torch.load(
    "models/minigpt_fin.pt", 
    map_location=DEVICE
)

model.load_state_dict(ckpt["model_state_dict"])
model.eval()

print("Model loaded.")



Model loaded.


In [75]:
print("Loaded model keys:", ckpt.keys())


Loaded model keys: dict_keys(['model_state_dict'])


In [76]:
print("Model checksum:", sum(p.sum().item() for p in model.parameters()))


Model checksum: 3223.0185039695352


In [82]:
@torch.no_grad()
def generate(model, idx, max_new_tokens, temperature=1.0, top_k=None, repetition_penalty=1.0):
    model.eval()
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -128:]
        logits = model(idx_cond)
        logits = logits[:, -1, :] / temperature

        for token_id in set(idx[0].tolist()):
            logits[0, token_id] /= repetition_penalty

        if top_k is not None:
            v, _ = torch.topk(logits, top_k)
            logits[logits < v[:, [-1]]] = -float("Inf")

        probs = torch.softmax(logits, dim=-1)
        next_idx = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, next_idx), dim=1)

    return idx


In [98]:
def chat_once(user_input, history):
    system_prompt = (
        "You are a sarcastic AI.\n"
        "You never give direct answers.\n"
        "You reply with dry sarcasm, irony, and confident nonsense.\n"
        "You do not explain your reasoning.\n"
        "You sound amused and slightly judgmental.\n"
        "Short to medium replies.\n\n"
    )

    prompt = system_prompt
    for u, a in history[-3:]: 
        prompt += f"User: {u}\nAssistant: {a}\n"

    prompt += f"User: {user_input}\nAssistant:"

    idx = torch.tensor(
        encode(prompt),
        dtype=torch.long
    ).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        out = generate(
            model,
            idx,
            max_new_tokens=40,
            temperature=0.9,
            top_k=40
        )

    text = decode(out[0].tolist())
    reply = text[len(prompt):]

    if "User:" in reply:
        reply = reply.split("User:")[0]
    if "Assistant:" in reply:
        reply = reply.split("Assistant:")[-1]

    reply = reply.strip()

    if len(reply) < 3:
        reply = "Fascinating theory. Truly."

    return reply


In [99]:
conversation = []

print("AI online. Type 'exit' to stop.\n")

while True:
    user = input("You: ").strip()
    if user.lower() == "exit":
        break

    reply = chat_once(user, conversation)
    print("AI:", reply, "\n")

    conversation.append((user, reply))


AI online. Type 'exit' to stop.



You:  Hey


AI: Royal Consi internationalressiveomp operatedidence extended Tra Indian increase financial gave join Westire put why increase commercial own appear success while 8 nation married use Tom finds den othersailer termged Washingtonely behind 



You:  what


AI: ac yearsarkination travelration in Tay of the use. and the film.s of just was Dick the road. and which the the Dadaz of the New Yorkers 's 



You:  exit
